In [ ]:
%%writefile requirements.txt
absl-py==2.2.2
altair==5.5.0
blinker==1.9.0
certifi==2025.4.26
charset-normalizer==3.3.2
click==8.1.8
Django==4.2.23
GitPython==3.1.45
idna==3.10
matplotlib==3.9.2
numpy==1.24.4
opencv-python==4.10.0
pandas==2.3.1
Pillow==10.3.0
protobuf==5.29.4
PyYAML==6.0.1
requests==2.32.3
scipy==1.13.1
seaborn==0.13.2
streamlit==1.47.1
torch==2.3.0
torchaudio==2.3.0
torchvision==0.16.0
tqdm==4.67.1
ultralytics==8.3.170


Writing requirements.txt


In [2]:
!pip install -r /content/requirements.txt

ERROR: Ignored the following yanked versions: 3.4.11.39, 3.4.17.61, 4.4.0.42, 4.4.0.44, 4.5.4.58, 4.5.5.62, 4.7.0.68
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement opencv-python==4.10.0 (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.46, 4.5.1.48, 4.5.3.56, 4.5.4.60, 4.5.5.64, 4.6.0.66, 4.7.0.72, 4.8.0.74, 4.8.0.76, 4.8.1.78, 4.9.0.80, 4.10.0.82, 4.10.0.84, 4.11.0.86, 4.12.0.88)
ERROR: No matching distribution found for opencv-python==4.10.0


In [9]:
!nvidia-smi

Tue Aug  5 00:32:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install -q streamlit ultralytics pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 857.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/

In [23]:
%%writefile app.py
import streamlit as st
from ultralytics import YOLO
from PIL import Image
import numpy as np
import cv2

# Load YOLOv12 model
MODEL_PATH = "/content/best.pt"
model = YOLO(MODEL_PATH)

# Function to draw highest confidence prediction
def draw_boxes(image, results, threshold):
    frame = np.array(image)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    predictions = []
    best_box = None
    best_conf = 0

    for result in results:
        for box in result.boxes:
            conf = float(box.conf[0])
            if conf > best_conf:
                best_box = box
                best_conf = conf

    if best_box and best_conf >= threshold:
        cls_id = int(best_box.cls[0])
        name = model.names[cls_id]
        label = f"{name} ({best_conf:.2f})"
        predictions.append(label)

        x1, y1, x2, y2 = map(int, best_box.xyxy[0])
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        thickness = 1
        (w, h), _ = cv2.getTextSize(label, font, font_scale, thickness)
        cv2.rectangle(frame, (x1, y1 - h - 5), (x1 + w, y1), (0, 255, 0), -1)
        cv2.putText(frame, label, (x1, y1 - 2), font, font_scale, (0, 0, 0), thickness)

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return frame, predictions

def main():
    st.title("🔍 YOLOv12 Object Detection App")
    st.write("Upload an image, select confidence threshold, and see predictions.")

    # Confidence threshold slider
    threshold = st.slider("Confidence Threshold", min_value=0.0, max_value=1.0, value=0.85, step=0.01)

    uploaded_file = st.file_uploader("📤 Upload an image", type=["jpg", "jpeg", "png", "bmp", "webp"])

    if uploaded_file:
        image = Image.open(uploaded_file).convert("RGB")
        results = model(image)

        annotated_image, predictions = draw_boxes(image, results, threshold)

        st.image(annotated_image, caption="🖼️ Detected Image", use_column_width=True)

        if predictions:
            st.subheader("📋 Predictions:")
            for pred in predictions:
                st.write(f"✅ {pred}")
        else:
            st.warning(f"❌ No object detected with confidence ≥ {threshold:.2f}")

if __name__ == "__main__":
    main()




Overwriting app.py


In [5]:
!pip install -q streamlit ultralytics
!npm install -g localtunnel




⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 3s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹

In [9]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared


--2025-08-05 02:17:09--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.7.0/cloudflared-linux-amd64 [following]
--2025-08-05 02:17:09--  https://github.com/cloudflare/cloudflared/releases/download/2025.7.0/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/37d2bad8-a2ed-4b93-8139-cbb15162d81d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-08-05T02%3A57%3A08Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-08-05T0

In [26]:
!nohup streamlit run /content/app.py &> streamlit.log &


In [27]:
!tail -n 10 streamlit.log





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.192.28.254:8501



In [ ]:
!./cloudflared tunnel --url http://localhost:8501




2025-08-05T03:10:31Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-08-05T03:10:31Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-08-05T03:10:34Z INF +--------------------------------------------------------------------------------------------+
2025-08-05T03:10:34Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-08-05T03:10:34Z INF |  https://rob-municipal-noon-merchants.trycloudflare.co